# Execute IBM Watson Tone Analyzer on Tweets

### Notebook Input
* Two JSON files of President Trump's tweets. 
 * First JSON file is 200 coronavirus-related tweets. 
 * Second JSON file is 200 non-coronavirus-related tweets

### Notebook Output
* A JSON file of 200 coronavirus-related tweets from President Trump
  * Each JSON object contains the tweet's ID, the tweet's Text, and IBM Watson Tone Analyzer results
* A JSON file of 200 non-coronavirus-related tweets
  * Each JSON object contains the tweet's ID, the tweet's Text, and IBM Watson Tone Analyzer results

In [ ]:
import json

In [ ]:
coronavirus_tweets     = []
non_coronavirus_tweets = []
    
with open('corona_virus_tweets_cleaned.json', encoding="utf8") as f:
  coronavirus_tweets = json.load(f)

with open('non_corona_virus_tweets_cleaned.json', encoding="utf8") as f:
  non_coronavirus_tweets = json.load(f)

print(f'coronavirus tweets count: {len(coronavirus_tweets)}')
print(f'non-coronavirus tweets count: {len(non_coronavirus_tweets)}')


In [ ]:
coronavirus_tone_analyzer_mapper     = []
non_coronavirus_tone_analyzer_mapper = []

for tweet in coronavirus_tweets:
    # write the tweet text to a temporary file
    # this was necessary to pass the text into the curl request below
    with open("temp_text.txt", "w", encoding="utf8") as outfile:
        outfile.write(tweet['text'])
    
    response = !curl -X POST -u "apikey:MY-KEY" --header "Content-Type: text/plain" --data-binary @\Users\netho\Desktop\TRUMP_TWEETS\trump-corona-sentiment\temp_text.txt "MY-URL"
    
    # retrieve the response, convert it into a dictionary
    tone_analyzer_response = json.loads(response[-1])
    # add the tweet's 'id_str' and 'text' attributes to the response for mapping and future reference
    tone_analyzer_response["id_str"] = tweet["id_str"]
    tone_analyzer_response["text"] = tweet["text"]
    # append to the list
    coronavirus_tone_analyzer_mapper.append(tone_analyzer_response)

# create the file
with open("coronavirus_tone_analyzer_mapper.json", "w", encoding="utf8") as outfile:
    json.dump(coronavirus_tone_analyzer_mapper, outfile)
    
##################################################
# Do the same thing for the non-coronavirus tweets
##################################################

for tweet in non_coronavirus_tweets:
    with open("temp_text.txt", "w", encoding="utf8") as outfile:
        outfile.write(tweet['text'])
    
    response = !curl -X POST -u "apikey:MY-KEY" --header "Content-Type: text/plain" --data-binary @\Users\netho\Desktop\TRUMP_TWEETS\trump-corona-sentiment\temp_text.txt "MY-URL"
    
    tone_analyzer_response = json.loads(response[-1])
    
    tone_analyzer_response["id_str"] = tweet["id_str"]
    tone_analyzer_response["text"] = tweet["text"]
    
    non_coronavirus_tone_analyzer_mapper.append(tone_analyzer_response)

with open("non_coronavirus_tone_analyzer_mapper.json", "w", encoding="utf8") as outfile:
    json.dump(non_coronavirus_tone_analyzer_mapper, outfile)

    
    

### Summary
* This notebook takes in two sets of tweets from President Donald Trump (one set of coronavirus-related tweets, and one set of non-coronavirus-related tweets)
* It passes each tweet through IBM Watson Tone Analyzer.  
* JSON files were created to save the results. The JSON files comprise of objects that include the IBM Watson Tone Analyzer results, as well as the tweet id for reference/mapping.